<a href="https://colab.research.google.com/github/manuelboi/MLsec_project/blob/main/MLsec_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Models:
1. Fixing Data Augmentation to Improve Adversarial Robustness
2. Robust Learning Meets Generative Models: Can Proxy Distributions Improve Adversarial Robustness?
3. MMA Training: Direct Input Space Margin Maximization through Adversarial Training

In [ ]:
%%capture
'''
!pip install git+https://github.com/fra31/auto-attack
!pip install adversarial-robustness-toolbox[pytorch]
!pip install secml
!pip install torch
'''
try:
  import robustbench
except:
  !pip install git+https://github.com/RobustBench/robustbench

from robustbench.utils import load_model
from robustbench.eval import benchmark
from robustbench.data import load_cifar10
from robustbench.model_zoo.enums import ThreatModel
#from autoattack import AutoAttack


Matteo: Fixing Data Augmentation to Improve Adversarial Robustness

In [ ]:
# hide entire cell output
%%capture

try:
  import smoothing
except:
  !git clone https://github.com/matteoturnu/smoothing.git
  # smoothing repository dependencies
  !conda create -n smoothing
  !conda activate smoothing
  !conda install pytorch torchvision cudatoolkit=10.0 -c pytorch
  !conda install scipy pandas statsmodels matplotlib seaborn
  !pip install setGPU

## Base model setup

In [ ]:
from smoothing.code.core import Smooth
import torch
import matplotlib.pyplot as plt


labels_dct = {0: "airplane", 1: "automobile", 2: "bird", 3: "cat", 4: "deer", 5: "dog", 6: "frog", 7: "horse", 8: "ship", 9: "truck"}
# use GPU if available, otherwise use CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Computing device used: ", device)

base_model = load_model(model_name='Rebuffi2021Fixing_70_16_cutmix_extra', dataset='cifar10', threat_model='L2')
base_model.to(device)
x_test, y_test = load_cifar10(10)
x_test, y_test = x_test.to(device), y_test.to(device)
x, y = x_test[2], y_test[2]

if device != "cpu":
  x = x.cpu()
x_image = x.permute(1, 2, 0)
plt.imshow(x_image)
print(f"True class: {y.item()} ({labels_dct[y.item()]})")



In [ ]:
# uncomment to delete and clone again the repository (test purposes)
"""
import os
os.system("rm -r smoothing")
!git clone https://github.com/matteoturnu/smoothing.git

# clear variable "device" (counteracting Colab fricking runtime problems)
del device
"""

## Smooth model setup
Training base model with Gaussian data augmentation: model learns on the perturbed images through Randomized Smoothing as well.

In [ ]:
try:
  import torchvision
except:
  !pip install torchvision

from smoothing.code.train_utils import AverageMeter, accuracy
from torchvision.datasets import CIFAR10

import torchvision.transforms as transforms
import time
from torch.utils.data import DataLoader, Subset
from torch.nn import CrossEntropyLoss
from torch.optim import SGD, Optimizer
from torch.optim.lr_scheduler import StepLR
import importlib.util
import re
from smoothing.code.train import train, test


transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

# Define the number of samples you want to take
num_samples = 2
# Create a subset of the dataset
indices = list(range(num_samples))

train_subset = Subset(trainset, indices)
test_subset = Subset(testset, indices)

trainloader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(test_subset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

#criterion = CrossEntropyLoss().cuda()
criterion = CrossEntropyLoss().to(device)
optimizer = SGD(base_model.parameters())
scheduler = StepLR(optimizer, step_size=30)

epochs = 1
sigma = 0.5     # gaussian noise standard deviation


for epoch in range(epochs):
  scheduler.step(epoch)
  before = time.time()
  train_loss, train_acc = train(trainloader, base_model, criterion, optimizer, epoch, sigma, device)
  test_loss, test_acc = test(testloader, base_model, criterion, epoch, sigma, device)
  after = time.time()

## Smooth model performances on perturbed images

In [ ]:
from smoothing.code.core import Smooth

n_classes = 10

alpha = 0.05   # (1 - alpha) is the confidence level (in this case is 0.95)
#n0 = 100        # number of samples for selection
n0 = 5
#n = 1000    # number of samples for estimation (certify) (too few samples but computation time is strongly affected with more)
n = 10

smooth_model = Smooth(base_model, n_classes, sigma)

# loading test samples (10000 images)
x_test, y_test = load_cifar10(10)
x_test, y_test = x_test.to(device), y_test.to(device)
print(x_test.shape)

top_classes = list()
for x, y in zip(x_test, y_test):
  top_class = smooth_model.predict(x, n0, alpha, batch_size=n0, device=device)
  top_class, radius = smooth_model.certify(x, n0, n, alpha, batch_size=n0, device=device)
  top_classes.append(top_class)

top_classes = torch.tensor(top_classes, dtype=torch.float64)
accuracy = torch.mean(top_classes == y, dtype=torch.float64)

print("Top classes: ", top_classes)
print("Y classes: ", y)
print("Accuracy on smooth model: ", accuracy)

In [10]:
# AutoAttack on model 1 with L2
model_1_L2 = base_model
benchmark(model_1_L2, threat_model=ThreatModel.L2, n_examples=1, eps=0.5)

KeyboardInterrupt: 

In [ ]:
# AutoAttack on model 1 with Linf
model_1_Linf = load_model(model_name='Rebuffi2021Fixing_70_16_cutmix_extra', dataset='cifar10', threat_model='Linf')
benchmark(model_1_Linf, threat_model=ThreatModel.Linf, n_examples=5, eps=8/255)

Manuel: Robust Learning Meets Generative Models: Can Proxy Distributions Improve Adversarial Robustness?

In [ ]:
# AutoAttack on model 2 with L2
model_2_L2 = load_model(model_name='Sehwag2021Proxy', dataset='cifar10', threat_model='L2')
benchmark(model_2_L2, threat_model=ThreatModel.L2, n_examples=5, eps=0.5)

In [ ]:
# AutoAttack on model 2 with Linf
model_2_Linf = load_model(model_name='Sehwag2021Proxy', dataset='cifar10', threat_model='Linf')
benchmark(model_2_Linf, threat_model=ThreatModel.Linf, n_examples=5, eps=8/255)

Elia: MMA Training: Direct Input Space Margin Maximization through Adversarial Training

In [ ]:
# Autoattack on model 3 with L2
model_3_L2 = load_model(model_name='Ding2020MMA', dataset='cifar10', threat_model='L2')
benchmark(model_3_L2, threat_model=ThreatModel.L2, n_examples=5, eps=0.5)

In [ ]:
# AutoAttack on model 3 with Linf
model_3_Linf = load_model(model_name='Sehwag2021Proxy', dataset='cifar10', threat_model='Linf')
benchmark(model_3_Linf, threat_model=ThreatModel.Linf, n_examples=5, eps=8/255)